In [1]:
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt

import numpy as np

import os

from PIL import Image

from loader import HandClassifyDataset

In [2]:
print(torch.cuda.device_count())

0


In [ ]:
hands_dataset = HandClassifyDataset(rootdir='./data/')

In [ ]:
handData = DataLoader(hands_dataset, batch_size=100, shuffle=True, num_workers=4)

In [ ]:
print(handData)

In [ ]:
for d in handData:
  print(d["image"].size(), d["hand"])
  break

## Check GPU availability

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Hyper parameters

In [ ]:
num_epochs = 10
num_classes = 2
batch_size = 100
learning_reate = 1e-3

## Model

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=2):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
      nn.Conv2d(),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    
    self.fc = nn.Linear()
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

In [ ]:
model = ConvNet(num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [ ]:
total_step = len(handData)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(handData):
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}'.
            format(epoch+1, num_epochs, i+1, total_step, loss.item()))

## Testing

In [ ]:
model.eval()

with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in handData:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
  print('Test Accuracy: {}%'.format(100 * correct / total))

In [ ]:
torch.save(model.state_dict(), 'model.ckpt')